<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/Softmax_mnist2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn, optim, cuda
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot 
import torch.nn.functional as F
import time

In [0]:
from torchvision import datasets, transforms

In [6]:
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


MNIST dataset (Train and Test)

In [3]:
train_dataset = datasets.MNIST(root= 'MNIST/processed/training.pt',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)


Extracting MNIST/processed/training.pt/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/processed/training.pt/MNIST/raw


Extracting MNIST/processed/training.pt/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/processed/training.pt/MNIST/raw



Extracting MNIST/processed/training.pt/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/processed/training.pt/MNIST/raw


Extracting MNIST/processed/training.pt/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/processed/training.pt/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [4]:
test_dataset = datasets.MNIST(root= 'MNIST/processed/test.pt',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)

Extracting MNIST/processed/test.pt/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/processed/test.pt/MNIST/raw




Extracting MNIST/processed/test.pt/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/processed/test.pt/MNIST/raw


Extracting MNIST/processed/test.pt/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/processed/test.pt/MNIST/raw


Extracting MNIST/processed/test.pt/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/processed/test.pt/MNIST/raw
Processing...
Done!


Data loader (input pipelines)

In [0]:
bs = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=bs, shuffle = True)
# it will give op as batch_idx, data & target
test_loader = DataLoader(dataset= test_dataset, batch_size= bs, shuffle = True)

In [6]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Trainng Minist model on {device}\n {"=" * 44}')

Trainng Minist model on cuda


In [0]:
class mnist(nn.Module):
  def __init__(self):
    super(mnist, self).__init__()
    self.l1 = nn.Linear(784, 520) #28*28 =784
    self.l2 = nn.Linear(520, 320)
    self.l3 = nn.Linear(320, 240)
    self.l4 = nn.Linear(240, 120)
    self.l5 = nn.Linear(120, 10)
    # self.softmax = F.softmax()
    # self.sigmoid = F.sigmoid()
    # self.relu = F.relu()

  def forward(self, data):
    # make image flat(n,1,28,28) then n*784 to layere1
    # x = data.reshape(1, 784)
    x= data.view(-1, 784)
    f1= self.relu(self.l1(x))
    f1= self.relu(self.l2(f1))
    f1= self.relu(self.l3(f1))
    f1= self.relu(self.l4(f1))
    # logit= self.relu(self.l5(f1))
    logit = self.l5(f1)
    return logit # is it required to have activation or not ans no



In [12]:
model = mnist()
model.to(device)

mnist(
  (l1): Linear(in_features=784, out_features=520, bias=True)
  (l2): Linear(in_features=520, out_features=320, bias=True)
  (l3): Linear(in_features=320, out_features=240, bias=True)
  (l4): Linear(in_features=240, out_features=120, bias=True)
  (l5): Linear(in_features=120, out_features=10, bias=True)
)

criterion and OPtimizer

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01, momentum =0.5)

Trainingdef

In [0]:
def train(epoch):
  model.train()
  for Epoch in range(5):
    for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      logit = model(data)
      loss = criterian(logit, target)
      loss.backward()
      optimizer.step()
      if batch_idx % 10 == 0:
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}% | Loss: {:.6f}', format(
            epoch, batch_idx*len(data), len(train_loader.dataset), 100. *batch_idx / len(train_loader), loss.item()
        ))


Testing

In [0]:
def test():
  model.eval()
  test_loss =0
  correct = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    #sum up the match loss
    test_loss += criterion(output, target).item()
    #get the index of the max value
    pred = output.data.max(1, keppdim=true)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print(f'====================\n test set : Average loss: {test_loss: .4f}, Accuracy: {correct}/{len(test_loader,dataset)} ' f'({100. * correct/len(test_loader.dataset):. 0f}%)')



In [26]:
if __name__ == '__main__':
  since = time.time()
  for epoch in range(1,10):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training timr: {m:.0f}m {s:.0f}s')
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Testing timr: {m:.0f}m {s:.0f}s')
  m, s = divmod(time.time() - since, 60)
  print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

RuntimeError: ignored